# 1. Thư viện

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import os

# 2. Thực thi

## 2.1. Crawl List Phone Link

In [29]:
def convert_price_to_int(price_str):
    # Chuyển đổi giá từ định dạng có dấu phân cách thành số nguyên
    price_str = price_str.replace('₫', '').replace('.', '').strip()
    try:
        return int(price_str)
    except ValueError:
        return None

def save_to_csv(data, filename):
    # Tạo DataFrame từ dữ liệu
    df = pd.DataFrame(data)
    
    # Kiểm tra nếu file đã tồn tại
    file_exists = os.path.isfile(filename)
    
    # Nếu file chưa tồn tại, tạo một file mới với tiêu đề cột cụ thể
    if not file_exists:
        # Tạo một DataFrame với các tiêu đề cột cụ thể
        df_empty = pd.DataFrame(columns=['ten_SanPham', 'giaTien', 'rating', 'comment', 'link_Phone'])
        # Ghi tiêu đề cột vào file CSV
        df_empty.to_csv(filename, index=False, encoding='utf-8')
    
    # Ghi dữ liệu vào file CSV
    df.to_csv(filename, mode='a', header=False, index=False, encoding='utf-8')

In [30]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

# Truy cập trang web
url = "https://www.thegioididong.com/dtdd"
driver.get(url)

time.sleep(3)

# Mở file CSV và viết tiêu đề nếu file không tồn tại
csv_file = '../data/data_List_Phone.csv'
save_to_csv([], csv_file)  # Tạo file CSV với tiêu đề nếu chưa có

In [31]:
# Nhấn nút "Xem thêm" để tải toàn bộ danh sách điện thoại
while True:
    try:
        # Kiểm tra xem nút "Xem thêm" có xuất hiện không
        view_more_button = driver.find_element(By.CLASS_NAME, "see-more-btn")
        
        # Nếu nút "Xem thêm" xuất hiện, nhấn vào nó
        if view_more_button.is_displayed():
            driver.execute_script("arguments[0].click();", view_more_button)
            time.sleep(2)  # Chờ trang tải thêm dữ liệu
        else:
            # Nếu không còn nút, thoát khỏi vòng lặp
            break
    except:
        # Thoát khỏi vòng lặp nếu không tìm thấy nút "Xem thêm"
        break

In [32]:
# Lấy toàn bộ sản phẩm từ danh sách
phone_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.listproduct li.item.ajaxed.__cate_42')

for phone in phone_elements:
    try:
        # Kiểm tra xem thẻ <div class="vote-txt"> có tồn tại không
        vote_txt = phone.find_element(By.CSS_SELECTOR, 'div.vote-txt')
        
        # Kiểm tra xem thẻ vote-txt có chứa text không
        if vote_txt.text.strip() != "":
            # Lấy tên sản phẩm
            ten_SanPham = phone.find_element(By.CSS_SELECTOR, 'a.main-contain h3').text
            
            # Lấy giá tiền
            giaTien_raw = phone.find_element(By.CSS_SELECTOR, 'strong.price').text
            giaTien = convert_price_to_int(giaTien_raw)
            
            # Lấy rating từ thẻ <b> và comment từ text ngoài thẻ <b>
            rating = float(vote_txt.find_element(By.TAG_NAME, 'b').text)
            # Lấy comment từ chuỗi đầy đủ của vote-txt và tách bỏ rating
            full_text = vote_txt.text
            comment = int(full_text.split('(')[-1].strip(')'))  # Lấy số trong dấu ngoặc ()
            
            # Lấy link sản phẩm từ thuộc tính href
            link_Phone = phone.find_element(By.CSS_SELECTOR, 'a.main-contain').get_attribute('href')
            
            # Lưu dữ liệu vào danh sách và ghi vào file CSV
            phone_data = {
                'ten_SanPham': ten_SanPham,
                'giaTien': giaTien,
                'rating': rating,
                'comment': comment,
                'link_Phone': link_Phone
            }
            save_to_csv([phone_data], csv_file)  # Ghi dữ liệu vào file CSV

    except Exception as e:
        print(f"Lỗi khi crawl dữ liệu cho sản phẩm: {e}")
        continue

# Đóng trình duyệt
driver.quit()

## 2.2. Crawl Detail Phone's Comments

In [33]:
# Đọc dữ liệu từ CSV
df = pd.read_csv('../data/data_List_Phone.csv')  # Đọc dữ liệu từ file CSV đã có

# Tạo dictionary để lưu thông tin sản phẩm
product_info = {
    row['link_Phone']: {
        'phone_Name': row['ten_SanPham'],
        'phone_Price': row['giaTien']
    }
    for _, row in df.iterrows()
}

In [34]:
# Tạo DataFrame trống để lưu trữ dữ liệu
df_result = pd.DataFrame(columns=["Product Name", "Current Price", "Reviewer", "Rating", "Comment", "Time Used"])

In [35]:
# Duyệt qua từng liên kết trong cột "link_Phone"
for link in product_info.keys():
    if pd.notna(link):  # Kiểm tra nếu link không phải là NaN
        phone_name = product_info[link]['phone_Name']
        phone_price = product_info[link]['phone_Price']
        page_Index = 1  # Bắt đầu lại với page_Index = 1 cho mỗi link mới

        driver = webdriver.Chrome()

        while True:
            # Thêm chuỗi "/danh-gia?page={page_Index}" vào cuối link
            modified_link = f"{link}/danh-gia?page={page_Index}"
            driver.get(modified_link)  # Mở liên kết đã chỉnh sửa
            time.sleep(3)  # Chờ vài giây để trang tải xong
            
            # Điều kiện cho page_Index từ 2 trở đi
            if page_Index > 1:
                try:
                    # Kiểm tra nếu thẻ <span class="active"> tồn tại bên trong <div class="pagcomment"> và <div class="pgrc">
                    active_span = driver.find_element(By.CSS_SELECTOR, "div.pgrc div.pagcomment span.active")
                except NoSuchElementException:
                    # Nếu không tìm thấy thẻ <span class="active">, dừng việc cào và chuyển sang link tiếp theo
                    break

            # Kiểm tra sự tồn tại của thẻ <ul class="comment-list">
            try:
                comment_list = driver.find_element(By.CSS_SELECTOR, "ul.comment-list")
                print("Thẻ <ul class='comment-list'> tồn tại.")
                
                # Duyệt qua từng thẻ <li class="par"> trong comment-list
                comments = driver.find_elements(By.CSS_SELECTOR, "li.par")
                temp_data = []  # Danh sách tạm thời để lưu dữ liệu từ các bình luận trên trang này

                for comment in comments:
                    try:
                        # 1. Lấy text từ thẻ <p class="cmt-top-name">
                        user_name = comment.find_element(By.CSS_SELECTOR, "div.cmt-top p.cmt-top-name").text
                        # Nếu user_name là chuỗi rỗng, gán nó thành "N/A"
                        if user_name == "":
                            user_name = "N/A"
                    except NoSuchElementException:
                        user_name = "N/A"

                    try:
                        # 2. Lấy số lượng thẻ <i class="iconcmt-starbuy">
                        user_rating = len(comment.find_elements(By.CSS_SELECTOR, "div.cmt-intro div.cmt-top-star i.iconcmt-starbuy"))
                    except NoSuchElementException:
                        user_rating = 0

                    try:
                        # 3. Lấy text từ thẻ <p class="cmt-txt">
                        user_comment = comment.find_element(By.CSS_SELECTOR, "div.cmt-content p.cmt-txt").text
                        # Nếu user_comment là chuỗi rỗng, gán nó thành "N/A"
                        if user_comment == "":
                            user_comment = "N/A"
                    except NoSuchElementException:
                        user_comment = "N/A"

                    try:
                        # 4. Lấy text từ thẻ <span class="cmtd dot-line">
                        user_time_comment = comment.find_element(By.CSS_SELECTOR, "div.cmt-command span.cmtd.dot-line").text
                        # Nếu user_time_comment là chuỗi rỗng, gán nó thành "N/A"
                        if user_time_comment == "":
                            user_time_comment = "N/A"
                        else:
                            # Tách các từ trong chuỗi
                            words = user_time_comment.split()
                            # Nếu có ít hơn 2 từ, sử dụng toàn bộ chuỗi
                            if len(words) < 2:
                                user_time_comment = "N/A"
                            else:
                                # Lấy 2 từ cuối cùng
                                user_time_comment = " ".join(words[-2:])
                    except NoSuchElementException:
                        user_time_comment = "N/A"

                    # Thêm dữ liệu vào danh sách tạm thời
                    temp_data.append({
                        'Product Name': phone_name,
                        'Current Price': phone_price,
                        'Reviewer': user_name,
                        'Rating': user_rating,
                        'Comment':  user_comment,
                        'Time Used': user_time_comment
                    })
                
                # Cập nhật DataFrame với dữ liệu từ trang này
                df_temp = pd.DataFrame(temp_data)
                df_result = pd.concat([df_result, df_temp], ignore_index=True)
                df_result.to_csv(r'../data/phone_data.csv', index=False, encoding='utf-8')  # Ghi dữ liệu vào file CSV

            except NoSuchElementException:
                print("Thẻ <ul class='comment-list'> không tồn tại.")
                break  # Nếu không tìm thấy thẻ comment-list, dừng lại và chuyển sang link kế tiếp
            
            # Tăng chỉ số page_Index
            page_Index += 1

        # Đóng trình duyệt sau khi hoàn thành
        driver.quit()

Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-list'> tồn tại.
Thẻ <ul class='comment-li